In [1]:
#importing needed libraries
import numpy as np
import pandas as pd
import json

# Data Cleaning

## Importing BRFSS responses

In [2]:
#creating a list for csv files for function generation
test_range = range(2009, 2019)

file_list = []

for x in test_range:
    
    file_front = 'BRFSS_'
    
    file_back = '{}.csv'.format(x)
    
    file_list.append(file_front + file_back) 

In [3]:
file_list

['BRFSS_2009.csv',
 'BRFSS_2010.csv',
 'BRFSS_2011.csv',
 'BRFSS_2012.csv',
 'BRFSS_2013.csv',
 'BRFSS_2014.csv',
 'BRFSS_2015.csv',
 'BRFSS_2016.csv',
 'BRFSS_2017.csv',
 'BRFSS_2018.csv']

In [4]:
#creating a list to store each dataframe

files = []

for file_n in file_list:
    
    files.append(pd.read_csv(file_n))

Wall time: 6min 42s


Now that our files are read in, a master data frame includeing each year's data needs to be generated.

In [5]:
Mega_BRFSS = pd.DataFrame()

Mega_BRFSS['Year'] = range(2009, 2019)

Mega_BRFSS['File_Name'] = file_list

Mega_BRFSS['File'] = files

Mega_BRFSS.set_index('Year', inplace=True)

Mega_BRFSS

Wall time: 5.35 s


In [6]:
for x in range(2009, 2019):
    
    print(Mega_BRFSS.at[x, 'File'].shape)

(432607, 405)
(451075, 397)
(506467, 454)
(475687, 359)
(491773, 336)
(464664, 279)
(441456, 330)
(486303, 275)
(450016, 358)
(437436, 275)


## BRFSS HIstorical Questions

In [7]:
#examing BRFSS Historical Questions
Questions = pd.read_csv('BRFSS_Hist_Qs.csv')

print(Questions.shape)

Questions.head()

(6353, 7)
Wall time: 2.79 s


In [8]:
#Dropping all questions before 2009
Questions = Questions[Questions.year >= 2009]

print(Questions.shape)

Questions.year.unique()

(2315, 7)


array([2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009],
      dtype=int64)

In [9]:
Questions.topic.unique()

array(['Health Status', 'Healthy Days — Health Related Quality of Life',
       'Health Care Access', 'Exercise', 'Inadequate Sleep',
       'Chronic Health Conditions', 'Oral Health', 'Demographics',
       'Tobacco Use', 'Alcohol Consumption', 'Immunization', 'Falls',
       'Seatbelt Use and Drinking and Driving',
       'Breast and Cervical Cancer Screening',
       'Prostate Cancer Screening', 'Colorectal Cancer Screening',
       'HIV/AIDS', 'Pre-Diabetes', 'Diabetes', 'Cognitive Decline',
       'Caregiver', 'E-Cigarettes', 'Marijuana Use', 'Sleep Disorder',
       'Depression and Anxiety', 'Respiratory Health (COPD Symptoms)',
       'Indoor Tanning', 'Excess Sun Exposure', 'Lung Cancer Screening',
       'Cancer Survivorship', 'Prostate Cancer Screening Decision Making',
       'Clinic Breast Exam',
       'Adult Human Papillomavirus (HPV) - Vaccination',
       'Tetanus Diphtheria (Tdap) (Adults)', 'Shingles (ZOSTAVAX or ZOS)',
       'Industry and Occupation',
       'Sexual

Too...many...topics. Need to remove factors unrelated to mental health of finances.

In [10]:
#list of variables being targeted for present project
target_words = ['Healthy Days', 'Anxiety', 'Health Care Access', 'Health Care Coverage', 'Depression', 
                'Mental', 'Demographics', 'Chronic Health']

In [11]:
#utilized assigned indicies to facilitate question removal
to_be_removed = []

for index,topic in enumerate(Questions.topic):
    
    checker = 0
    
    for word in target_words:
        
        if word in topic:
            
            checker += 1
            
    if checker == 0:
        
        to_be_removed.append(index)

In [12]:
#checking the list of indexes for accuracy
to_be_removed[0:6]

[0, 8, 9, 23, 24, 51]

In [13]:
#creating final list of wanted questions
Questions = Questions.drop(to_be_removed)

In [14]:
#confirming isolation of desired topics
topics = Questions.topic.unique()

topics = topics.tolist() 

topics

['Healthy Days — Health Related Quality of Life',
 'Health Care Access',
 'Chronic Health Conditions',
 'Demographics',
 'Depression and Anxiety',
 'Healthy Days (Symptoms)',
 'Anxiety and Depression',
 'Health Status/Healthy Days',
 'Health Care Coverage/Access',
 'Quality Of Life/Healthy Days (Symptoms)/Disability',
 'Mental Illness and Stigma']

In [15]:
#SKIP SKIP SKIP SKIP SKIP SKIP#
#identifying specific questions to be removed from dataset
#on_the_bubble = []
#for question in Questions.question.unique():
    #choice = input(question + ' ' + 'Keep or Delete')
    #if choice == 'd':
        #on_the_bubble.append(question)
#with open('on_the_bubble.json', 'w') as json_file:
    #json.dump(on_the_bubble, json_file)

The above cell is meant to tailer our data around pertinent questions within the survey. 

In [21]:
#saved results so on_the_bubble doesn't need to be executed multiple times

with open('C:/Users/Desmond/DS_Project_Portfolio/Data_Cache/BRFSS_Data/on_the_bubble.json') as f:
    
    on_the_bubble = json.load(f)

In [22]:
#removing unnecesary questions

for question in Questions.question:
    
    for o in on_the_bubble:
        
        if question == o:
            
            Questions = Questions[Questions.question != o] 

In [23]:
#Making questions uniform for easier analysis

for variable in Questions.variablename.unique():
    
    options = Questions.loc[Questions.variablename==variable].question.unique()
    
    if len(options) == 1:
        
        continue
        
    if len(options) > 1:
        
        keep = options[0]
        
        Questions['question'] = Questions['question'].replace(options, keep)

In [24]:
Questions.replace({'variablename': {'ORACE2': 'ORACE3'}}, inplace=True)

In [25]:
#updating ORACE2 to ORACE3 for uniformity, NaNs were replaced with 99

for file in files:
    
    df = file
    
    for column in df.columns:
        
        if column =='ORACE2':
            
            df.rename(columns={'ORACE2': 'ORACE3'}, inplace=True)
            
        elif column == 'ORACE3':
            
            df.replace({'ORACE3': {1: 10, 2:20, 3:30, 4:40, 5:50, 6:60, 7:77, 8:88, 9:99}}, inplace=True)
            
            df['ORACE3'].fillna(99, inplace=True)
            
            print(x, df.ORACE3.unique())
            
        else:
            
            continue

In [26]:
for response in Questions.responses.unique():
    
    options_r = Questions.loc[Questions.variablename== 'ORACE3'].responses.unique()
    
    if len(options_r) == 1:
        
        continue
        
    if len(options_r) > 1:
        
        keep = options_r[0]
        
        print(keep)
        
        Questions['responses'] = Questions['responses'].replace(options_r, keep)

10=White 20=Black or African American 30=American Indian or Alaska Native 40=Asian 41=Asian Indian 42=Chinese 43=Filipino 44=Japanese 45=Korean 46=Vietnamese 47=Other Asian 50=Pacific Islander 51=Native Hawaiian 52=Guamanian or Chamorro 53=Samoan 54=Other Pacific Islander 60=Other 77=Don´t know/Not Sure 99=Refused


In [27]:
#making AGE variable uniform across study years

for response in Questions.responses.unique():
    
    options_r = Questions.loc[Questions.variablename== 'AGE'].responses.unique()
    
    if len(options_r) == 1:
        
        continue
        
    if len(options_r) > 1:
        
        keep = options_r[1]
        
        Questions['responses'] = Questions['responses'].replace(options_r, keep)

In [28]:
display(Questions.loc[Questions.variablename== 'AGE'].responses.unique())

display(Questions[Questions.variablename == 'AGE'])

array(['_ _=Code age in years  0 7=DK/NS   0 9=Refused'], dtype=object)

,topic,question,variablename,responses,year,type,displayorder
26,Demographics,What is your age?,AGE,_ _=Code age in years 0 7=DK/NS 0 9=Refused,2018,Core Question,27
217,Demographics,What is your age?,AGE,_ _=Code age in years 0 7=DK/NS 0 9=Refused,2017,Core Question,32
443,Demographics,What is your age?,AGE,_ _=Code age in years 0 7=DK/NS 0 9=Refused,2016,Core Question,27
636,Demographics,What is your age?,AGE,_ _=Code age in years 0 7=DK/NS 0 9=Refused,2015,Core Question,28
844,Demographics,What is your age?,AGE,_ _=Code age in years 0 7=DK/NS 0 9=Refused,2014,Core Question,26
1026,Demographics,What is your age?,AGE,_ _=Code age in years 0 7=DK/NS 0 9=Refused,2013,Core Question,27
1224,Demographics,What is your age?,AGE,_ _=Code age in years 0 7=DK/NS 0 9=Refused,2012,Core Question,25
1460,Demographics,What is your age?,AGE,_ _=Code age in years 0 7=DK/NS 0 9=Refused,2011,Core Question,32
1760,Demographics,What is your age?,AGE,_ _=Code age in years 0 7=DK/NS 0 9=Refused,2010,Core Question,27
2033,Demographics,What is your age?,AGE,_ _=Code age in years 0 7=DK/NS 0 9=Refused,2009,Core Question,27


# Cleaning

In [29]:
target_variables = (Questions.variablename.unique()).tolist()

In [30]:
target = Questions[Questions.year == 2009]

In [31]:
target_variables = target.variablename.unique().tolist()

In [32]:
needed_variables = ['_STATE', '_GEOSTR', '_DENSTR2']

In [33]:
for variable in needed_variables:
    
    target_variables.append(variable)

In [34]:
for x in range(2009, 2019):
    
    file = Mega_BRFSS.at[x, 'File']
    
    target = Questions[Questions.year == x]
    
    target_variables = target.variablename.unique().tolist()
    
    for variable in needed_variables:
        
        target_variables.append(variable)
        
    for column in file.columns:
        
        if column in target_variables:
            
            continue
            
        else:
            
            file.drop(column, axis=1, inplace=True) 
            
    print(file.shape)

(432607, 36)
(451075, 35)
(506467, 36)
(475687, 36)
(491773, 42)
(464664, 34)
(441456, 34)
(486303, 33)
(450016, 33)
(437436, 32)


We need to identify the prevalence of variables across study years to determine if there is enough data to represent the populace. 

In [35]:
empty = {}
for file in files: 
    
    for counter,column in enumerate(file.columns):
        
        #print(x, counter, column)
        
        if column in empty:
            
            empty[column] += 1
            
        else:
            
            empty[column] = 1
            
empty

{'_STATE': 10,
 '_GEOSTR': 4,
 '_DENSTR2': 4,
 'GENHLTH': 5,
 'PHYSHLTH': 10,
 'MENTHLTH': 10,
 'POORHLTH': 10,
 'HLTHPLAN': 2,
 'PERSDOC2': 10,
 'MEDCOST': 10,
 'CHECKUP1': 10,
 'AGE': 4,
 'HISPANC2': 4,
 'MRACE': 4,
 'ORACE3': 4,
 'MARITAL': 10,
 'CHILDREN': 10,
 'EDUCA': 10,
 'INCOME2': 10,
 'CTYCODE': 2,
 'SEX': 9,
 'QLACTLM2': 3,
 'PAINACT2': 9,
 'QLMENTL2': 9,
 'QLSTRES2': 9,
 'QLHLTH2': 9,
 'MISNERVS': 3,
 'MISHOPLS': 3,
 'MISRSTLS': 3,
 'MISDEPRD': 3,
 'MISEFFRT': 3,
 'MISWTLES': 3,
 'MISNOWRK': 3,
 'MISTMNT': 5,
 'MISTRHLP': 3,
 'MISPHLPF': 3,
 'ADPLEASR': 3,
 'ADDOWN': 3,
 'ADSLEEP': 3,
 'ADENERGY': 3,
 'ADEAT1': 3,
 'ADFAIL': 3,
 'ADTHINK': 3,
 'ADMOVE': 3,
 'ADANXEV': 3,
 'ADDEPEV': 1,
 'HLTHPLN1': 8,
 'ADDEPEV2': 8,
 'CTYCODE1': 2,
 'INTERNET': 5,
 'DECIDE': 6,
 'DIFFALON': 6,
 'MEDICARE': 5,
 'HLTHCVRG': 1,
 'DELAYMED': 4,
 'DLYOTHER': 5,
 'NOCOV121': 5,
 'LSTCOVRG': 5,
 'DRVISITS': 5,
 'MEDSCOST': 3,
 'CARERCVD': 5,
 'MEDBILLS': 1,
 'CPDEMO1': 3,
 'RENTHOM1': 5,
 'HLTHCV

It looks like there are a lot of variables that do not appear across all questionnaires. Variables appearing at least half the time will be included. 

In [36]:
final_round = []

for key, value in empty.items():
    
    if value >=5:
        
        print(key, value)
        
        final_round.append(key)

_STATE 10
GENHLTH 5
PHYSHLTH 10
MENTHLTH 10
POORHLTH 10
PERSDOC2 10
MEDCOST 10
CHECKUP1 10
MARITAL 10
CHILDREN 10
EDUCA 10
INCOME2 10
SEX 9
PAINACT2 9
QLMENTL2 9
QLSTRES2 9
QLHLTH2 9
MISTMNT 5
HLTHPLN1 8
ADDEPEV2 8
INTERNET 5
DECIDE 6
DIFFALON 6
MEDICARE 5
DLYOTHER 5
NOCOV121 5
LSTCOVRG 5
DRVISITS 5
CARERCVD 5
RENTHOM1 5


In [37]:
# DLYOTHER removed due to inconsistencies. 
final_round.remove('DLYOTHER')

We need to examine the way questions were asked accross questionnaires. If not uniform, data tranformation will be necessary for uniformity. 

In [38]:
for x in final_round:
    
    if x == '_STATE':
        
        pass
    
    else:
        
        if len(Questions[Questions.variablename ==x].responses.unique()) > 1:
            
            print(x, '\n', Questions[Questions.variablename ==x].responses.unique(), '\n')

            options = Questions[Questions.variablename ==x].responses.unique()
            
            keep = options[0]
            
            Questions['responses'] = Questions['responses'].replace(options, keep)

PHYSHLTH 
 ['1-30=Number of days 88=None 77=Don’t know/Not sure 99=Refused'
 '__ __=Number of days  88=None  77=DK/NS  99=Refused'] 

PERSDOC2 
 ['1=Yes, only one 2=More than one 3=No 7=Don’t know/Not Sure 9=Refused'
 '1=Yes, only one  2=More than one  3=No  7=DK/NS  9=Refused'] 

MEDCOST 
 ['1=Yes 2=No 7=Don’t know/Not sure 9=Refused'
 '1=Yes  2=No  7=DK/NS  9=Refused'] 

CHECKUP1 
 ['1=Within past year (anytime less than 12 months ago) 2=Within past 2 years (1 year but less than 2 years ago) 3=Within past 5 years (2 years but less than 5 years ago) 4=5 or more years ago 7=Don’t know/Not sure 8=Never 9=Refused'
 '1=Within past year (anytime less than 12 months ago)  2=Within past 2 years (1 year but less than 2 years ago)  3=Within past 5 years (2 years but less than 5 years ago)   4=5 or more years ago  7=DK/NS  8=Never  9=Refused'] 

MARITAL 
 ['1=Married 2=Divorced 3=Widowed 4=Separated 5=Never married 6=A member of an unmarried couple 9=Refused'
 '1=Married  2=Divorced  3=Widowed 

In [39]:
for x in range(2009, 2019):
    
    holder = Mega_BRFSS.at[x, 'File']
    
    for column in holder.columns:
        
        if column in final_round:
            
            continue
            
        else:
            
            holder.drop(column, axis=1, inplace=True) 

In [40]:
for x in range(2009, 2019):
    
    print(Mega_BRFSS.at[x, 'File'].shape)

(432607, 18)
(451075, 17)
(506467, 20)
(475687, 20)
(491773, 28)
(464664, 27)
(441456, 23)
(486303, 27)
(450016, 27)
(437436, 21)


In [41]:
for x in range(2009, 2019):
    
    holder = Mega_BRFSS.at[x, 'File']
     
    for column in holder.columns:
        
        for spot in final_round:
            
            if spot == column:
                
                print(column, '\n',Questions[Questions.variablename ==spot].responses.unique(), '\n', 
                      holder[column].unique(), '\n')
                
                if 99 in holder[column].unique():
                    
                    Mega_BRFSS.at[x, 'File'][column].fillna(99, inplace=True)
                    
                    print('After fix:', column, '\n', Mega_BRFSS.at[x, 'File'][column].unique(), '\n')
                    
                elif 9 in holder[column].unique():
                    
                    Mega_BRFSS.at[x, 'File'][column].fillna(9, inplace=True)
                    
                    print(column, '\n', Mega_BRFSS.at[x, 'File'][column].unique(), '\n')
                    
                else:
                    
                    pass

_STATE 
 [] 
 [ 1.  2.  4.  5.  6.  8.  9. 10. 11. 12. 13. 15. 16. 17. 18. 19. 20. 21.
 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35. 36. 37. 38. 39.
 40. 41. 42. 44. 45. 46. 47. 48. 49. 50. 51. 53. 54. 55. 56. 66. 72. 78.] 

_STATE 
 [ 1.  2.  4.  5.  6.  8.  9. 10. 11. 12. 13. 15. 16. 17. 18. 19. 20. 21.
 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35. 36. 37. 38. 39.
 40. 41. 42. 44. 45. 46. 47. 48. 49. 50. 51. 53. 54. 55. 56. 66. 72. 78.] 

GENHLTH 
 ['1=Excellent  2=Very good  3=Good  4=Fair  5=Poor  7=DK/NS  9=Refused'] 
 [ 3.  4.  5.  2.  1.  7.  9. nan] 

GENHLTH 
 [3. 4. 5. 2. 1. 7. 9.] 

PHYSHLTH 
 ['1-30=Number of days 88=None 77=Don’t know/Not sure 99=Refused'] 
 [88. 15.  7. 30.  4. 77.  2.  1.  3. 10. 23. 21. 20. 14.  5. 12.  8.  6.
 22. 25. 29. 11. 18. 99. 27. 28.  9. 24. 16. nan 17. 13. 19. 26.] 

After fix: PHYSHLTH 
 [88. 15.  7. 30.  4. 77.  2.  1.  3. 10. 23. 21. 20. 14.  5. 12.  8.  6.
 22. 25. 29. 11. 18. 99. 27. 28.  9. 24. 16. 17. 13. 19. 26.] 

_STATE 
 [ 1.  2.  4.  5.  6.  8.  9. 10. 11. 12. 13. 15. 16. 17. 18. 19. 20. 21.
 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35. 36. 37. 38. 39.
 40. 41. 42. 44. 45. 46. 47. 48. 49. 50. 51. 53. 54. 55. 56. 66. 72.] 

GENHLTH 
 ['1=Excellent  2=Very good  3=Good  4=Fair  5=Poor  7=DK/NS  9=Refused'] 
 [ 4.  2.  3.  5.  1.  7.  9. nan] 

GENHLTH 
 [4. 2. 3. 5. 1. 7. 9.] 

PHYSHLTH 
 ['1-30=Number of days 88=None 77=Don’t know/Not sure 99=Refused'] 
 [88. 12. 25. 30.  1. 10. 28. 20. 14.  5.  2. 77. 99. 15.  7.  3.  4.  6.
 11. 21.  8.  9. 16. 27. 23. 29. 18. 26. 13. 22. 17. 24. 19. nan] 

After fix: PHYSHLTH 
 [88. 12. 25. 30.  1. 10. 28. 20. 14.  5.  2. 77. 99. 15.  7.  3.  4.  6.
 11. 21.  8.  9. 16. 27. 23. 29. 18. 26. 13. 22. 17. 24. 19.] 

MENTHLTH 
 ['1-30=Number of days 88=None 77=Don’t know/Not sure 99=Refused'] 
 [30.  4.  3. 88. 15.  2.  5. 14. 10. 99.  7. 28. 20. 25.  1. 26.  6. 77.
 12.  9. 29. 18. 21.  8. 27. 16. 17. 24. 22. 23. 11. 13. 19.] 

After fix: MENTHLTH 
 

MISTMNT 
 ['1=Yes 2=No 7=Don’t know/Not sure 9=Refused'] 
 [nan  2.  1.  9.  7.] 

MISTMNT 
 [9. 2. 1. 7.] 

_STATE 
 [] 
 [ 1.  2.  4.  5.  6.  8.  9. 10. 11. 12. 13. 15. 16. 17. 18. 19. 20. 21.
 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35. 36. 37. 38. 39.
 40. 41. 42. 44. 45. 46. 47. 48. 49. 50. 51. 53. 54. 55. 56. 66. 72.] 

_STATE 
 [ 1.  2.  4.  5.  6.  8.  9. 10. 11. 12. 13. 15. 16. 17. 18. 19. 20. 21.
 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35. 36. 37. 38. 39.
 40. 41. 42. 44. 45. 46. 47. 48. 49. 50. 51. 53. 54. 55. 56. 66. 72.] 

GENHLTH 
 ['1=Excellent  2=Very good  3=Good  4=Fair  5=Poor  7=DK/NS  9=Refused'] 
 [ 4.  3.  2.  1.  5.  7.  9. nan] 

GENHLTH 
 [4. 3. 2. 1. 5. 7. 9.] 

PHYSHLTH 
 ['1-30=Number of days 88=None 77=Don’t know/Not sure 99=Refused'] 
 [30. 88.  3.  2. 10.  1.  5. 77. 14.  4. 23. 99. 15. 25. 20. 28.  7.  8.
 26. 22. 12. 11. 21. 17.  6.  9. 29. 13. 16. 19. 18. 27. 24. nan] 

After fix: PHYSHLTH 
 [30. 88.  3.  2. 10.  1.  5. 77. 1

ADDEPEV2 
 [2. 1. 7. 9.] 

MARITAL 
 ['1=Married 2=Divorced 3=Widowed 4=Separated 5=Never married 6=A member of an unmarried couple 9=Refused'] 
 [ 1.  3.  2.  4.  5.  6.  9. nan] 

MARITAL 
 [1. 3. 2. 4. 5. 6. 9.] 

CHILDREN 
 ['1-87=Number of children 88=None 99=Refused'] 
 [88.  1.  2.  3.  5.  4. nan  6. 99.  7.  8. 14. 10.  9. 11. 77. 12. 17.
 24. 15. 18. 25. 22. 13.] 

After fix: CHILDREN 
 [88.  1.  2.  3.  5.  4. 99.  6.  7.  8. 14. 10.  9. 11. 77. 12. 17. 24.
 15. 18. 25. 22. 13.] 

EDUCA 
 ['1=Never attended school or only kindergarten 2=Grades 1 through 8 (Elementary) 3=Grades 9 through 11 (Some high school) 4=Grade 12 or GED (High school graduate) 5=College 1 year to 3 years (Some college or technical school) 6=College 4 years or more (College graduate) 9=Refused'] 
 [ 5.  4.  6.  3.  2.  1.  9. nan] 

EDUCA 
 [5. 4. 6. 3. 2. 1. 9.] 

INCOME2 
 ['1=Less than $10,000 2=Less than $15,000 ($10,000 to less than $15,000) 3=Less than $20,000 ($15,000 to less than $20,000) 4=Less 

After fix: INCOME2 
 [ 3.  1. 99.  8. 77.  6.  4.  7.  5.  2.] 

INTERNET 
 ['1=Yes 2=No 7=Don’t know/Not Sure 9=Refused'] 
 [ 2.  1. nan  9.  7.] 

INTERNET 
 [2. 1. 9. 7.] 

DECIDE 
 ['1=Yes 2=No 7=Don’t know/Not Sure 9=Refused'] 
 [ 2.  1. nan  9.  7.] 

DECIDE 
 [2. 1. 9. 7.] 

DIFFALON 
 ['1=Yes 2=No 7=Don’t know/Not Sure 9=Refused'] 
 [ 1.  2. nan  7.  9.] 

DIFFALON 
 [1. 2. 9. 7.] 

PAINACT2 
 ['1-30=Number of days 88=None 77=Don’t know/Not sure 99=Refused'] 
 [nan] 

QLMENTL2 
 ['1-30=Number of days 88=None 77=Don’t know/Not sure 99=Refused'] 
 [nan] 

QLSTRES2 
 ['1-30=Number of days 88=None 77=Don’t know/Not sure 99=Refused'] 
 [nan] 

QLHLTH2 
 ['1-30=Number of days 88=None 77=Don’t know/Not sure 99=Refused'] 
 [nan] 

MISTMNT 
 ['1=Yes 2=No 7=Don’t know/Not sure 9=Refused'] 
 [nan  2.  1.  7.  9.] 

MISTMNT 
 [9. 2. 1. 7.] 

_STATE 
 [] 
 [ 1.  2.  4.  5.  6.  8.  9. 10. 11. 12. 13. 15. 16. 17. 18. 19. 20. 21.
 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35. 36. 37

After fix: MENTHLTH 
 [88. 25.  1. 77. 30. 15.  3.  2. 10. 14.  5.  6.  7. 12.  4. 20. 99. 28.
  8. 22.  9. 23. 13. 27. 19. 16. 26. 21. 18. 29. 17. 24. 11.] 

POORHLTH 
 ['1-30=Number of days 88=None 77=Don’t know/Not sure 99=Refused'] 
 [nan 14. 88.  4. 77. 30.  1. 15.  5.  3.  6.  2. 20.  7. 10. 99. 25. 28.
 11. 12. 21.  8. 13. 27. 19.  9. 18. 17. 16. 29. 23. 24. 26. 22.] 

After fix: POORHLTH 
 [99. 14. 88.  4. 77. 30.  1. 15.  5.  3.  6.  2. 20.  7. 10. 25. 28. 11.
 12. 21.  8. 13. 27. 19.  9. 18. 17. 16. 29. 23. 24. 26. 22.] 

HLTHPLN1 
 ['1=Yes 2=No 7=Don’t know/Not Sure 9=Refused'] 
 [ 1.  2.  9.  7. nan] 

HLTHPLN1 
 [1. 2. 9. 7.] 

PERSDOC2 
 ['1=Yes, only one 2=More than one 3=No 7=Don’t know/Not Sure 9=Refused'] 
 [ 1.  2.  3.  9.  7. nan] 

PERSDOC2 
 [1. 2. 3. 9. 7.] 

MEDCOST 
 ['1=Yes 2=No 7=Don’t know/Not sure 9=Refused'] 
 [ 2.  1.  7.  9. nan] 

MEDCOST 
 [2. 1. 7. 9.] 

CHECKUP1 
 ['1=Within past year (anytime less than 12 months ago) 2=Within past 2 years (1 year bu

After fix: INCOME2 
 [ 6.  4.  3. 99.  8.  5.  2.  7. 77.  1.] 

DECIDE 
 ['1=Yes 2=No 7=Don’t know/Not Sure 9=Refused'] 
 [ 2.  1.  7. nan  9.] 

DECIDE 
 [2. 1. 7. 9.] 

DIFFALON 
 ['1=Yes 2=No 7=Don’t know/Not Sure 9=Refused'] 
 [ 2.  1. nan  7.  9.] 

DIFFALON 
 [2. 1. 9. 7.] 

MEDICARE 
 ['1=Yes 2=No 7=Don’t know/Not sure 9=Refused'] 
 [nan  2.  9.  1.  7.] 

MEDICARE 
 [9. 2. 1. 7.] 

NOCOV121 
 ['1=Yes 2=No 7=Don’t know/Not Sure 9=Refused'] 
 [nan  2.  1.  7.  9.] 

NOCOV121 
 [9. 2. 1. 7.] 

LSTCOVRG 
 ['1=6 months or less 2=More than 6 months, but not more than 1 year ago 3=More than 1 year, but not more than 3 years ago 4=More than 3 years 5=Never 7=Don’t know/Not Sure 9=Refused'] 
 [nan  2.  5.  7.  4.  3.  1.  9.] 

LSTCOVRG 
 [9. 2. 5. 7. 4. 3. 1.] 

DRVISITS 
 ['1-76=Number of times 88=None 77=Don’t know/Not Sure 99=Refused'] 
 [nan  3. 77.  5. 88.  4.  2. 99.  6.  1. 12. 30. 20. 15.  8.  7.  9. 10.
 14. 76. 40. 25. 16. 24. 19. 36. 21. 50. 35. 18. 75. 28. 11. 17. 48. 23.


In [42]:
for x in range(2009, 2019):
    
    holder = Mega_BRFSS.at[x, 'File']
    
    percent_missing = holder.isnull().sum()*100/len(holder)
    
    print('\n', x, '\n',percent_missing)


 2009 
 _STATE        0.0
GENHLTH       0.0
PHYSHLTH      0.0
MENTHLTH      0.0
POORHLTH      0.0
PERSDOC2      0.0
MEDCOST       0.0
CHECKUP1      0.0
MARITAL       0.0
CHILDREN      0.0
EDUCA         0.0
INCOME2       0.0
SEX           0.0
PAINACT2    100.0
QLMENTL2    100.0
QLSTRES2    100.0
QLHLTH2     100.0
MISTMNT       0.0
dtype: float64

 2010 
 _STATE      0.0
GENHLTH     0.0
PHYSHLTH    0.0
MENTHLTH    0.0
POORHLTH    0.0
PERSDOC2    0.0
MEDCOST     0.0
CHECKUP1    0.0
MARITAL     0.0
CHILDREN    0.0
EDUCA       0.0
INCOME2     0.0
SEX         0.0
PAINACT2    0.0
QLMENTL2    0.0
QLSTRES2    0.0
QLHLTH2     0.0
dtype: float64

 2011 
 _STATE        0.0
GENHLTH       0.0
PHYSHLTH      0.0
MENTHLTH      0.0
POORHLTH      0.0
HLTHPLN1      0.0
PERSDOC2      0.0
MEDCOST       0.0
CHECKUP1      0.0
ADDEPEV2      0.0
MARITAL       0.0
CHILDREN      0.0
EDUCA         0.0
INCOME2       0.0
SEX           0.0
PAINACT2    100.0
QLMENTL2    100.0
QLSTRES2    100.0
QLHLTH2     100.0
MISTM

In [43]:
for file in files:
    
    print(file.shape)

(432607, 18)
(451075, 17)
(506467, 20)
(475687, 20)
(491773, 28)
(464664, 27)
(441456, 23)
(486303, 27)
(450016, 27)
(437436, 21)


In [44]:
BRFSS_FINAL = pd.concat(files)

Wall time: 3.32 s


In [45]:
for column in BRFSS_FINAL:
    
    if 99. in BRFSS_FINAL[column].unique():
        
        BRFSS_FINAL[column].fillna(99., inplace=True)
        
    else:
        
        BRFSS_FINAL[column].fillna(9., inplace=True)

In [46]:
# Removing countries outside of the continuous United States to declutter

BRFSS_FINAL = BRFSS_FINAL[BRFSS_FINAL['_STATE'] < 66]

BRFSS_FINAL

,_STATE,GENHLTH,PHYSHLTH,MENTHLTH,POORHLTH,PERSDOC2,MEDCOST,CHECKUP1,MARITAL,CHILDREN,...,ADDEPEV2,INTERNET,DECIDE,DIFFALON,MEDICARE,NOCOV121,LSTCOVRG,DRVISITS,CARERCVD,RENTHOM1
0,1.0,3.0,88.0,88.0,99.0,1.0,2.0,1.0,1.0,88.0,...,9.0,9.0,9.0,9.0,9.0,9.0,9.0,99.0,9.0,9.0
1,1.0,4.0,15.0,88.0,88.0,1.0,2.0,1.0,3.0,88.0,...,9.0,9.0,9.0,9.0,9.0,9.0,9.0,99.0,9.0,9.0
2,1.0,5.0,7.0,88.0,77.0,1.0,2.0,1.0,1.0,88.0,...,9.0,9.0,9.0,9.0,9.0,9.0,9.0,99.0,9.0,9.0
3,1.0,5.0,30.0,30.0,30.0,1.0,2.0,1.0,2.0,88.0,...,9.0,9.0,9.0,9.0,9.0,9.0,9.0,99.0,9.0,9.0
4,1.0,2.0,4.0,15.0,88.0,1.0,2.0,1.0,4.0,88.0,...,9.0,9.0,9.0,9.0,9.0,9.0,9.0,99.0,9.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430944,56.0,9.0,88.0,88.0,99.0,1.0,2.0,1.0,1.0,2.0,...,2.0,9.0,2.0,2.0,9.0,9.0,9.0,99.0,9.0,1.0
430945,56.0,9.0,88.0,88.0,99.0,1.0,2.0,1.0,1.0,88.0,...,2.0,9.0,2.0,1.0,9.0,9.0,9.0,99.0,9.0,1.0
430946,56.0,9.0,88.0,88.0,99.0,1.0,2.0,1.0,1.0,1.0,...,2.0,9.0,2.0,2.0,9.0,9.0,9.0,99.0,9.0,1.0
430947,56.0,9.0,1.0,88.0,88.0,3.0,2.0,1.0,1.0,1.0,...,2.0,9.0,2.0,2.0,9.0,9.0,9.0,99.0,9.0,1.0


In [47]:
BRFSS_FINAL.to_csv('BRFSS_FINAL', index=None)
Questions.to_csv('Questions', index=None)